In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
# Cargar los datos desde los archivos CSV
transaction = pd.read_csv("./data/transaction_df.csv")
inventory = pd.read_csv("./data/inventory_df.csv")
transaction_df = transaction.copy()
inventory_df = inventory.copy()

In [ ]:
# # Convertir la columna 'fecha' a tipo datetime
transaction_df['fecha'] = pd.to_datetime(transaction_df['fecha'])
inventory_df['fecha'] = pd.to_datetime(inventory_df['fecha'], format='%Y_%m_%d')


In [ ]:

# # Filtrar las transacciones en días con números pares
transaction_df_even = transaction_df[transaction_df['fecha'].dt.day % 2 == 0]

# # Agrupar por usuario y calcular el total gastado por usuario
gasto_por_usuario = transaction_df_even.groupby('id_usuario')['total_pagado_usd'].sum()

# # Obtener los 10 usuarios que gastaron más
top_10_usuarios = gasto_por_usuario.nlargest(10)

print(top_10_usuarios)

In [ ]:
transaction_df['fecha'].min()

In [ ]:
# Obtener el precio promedio por producto por carrito comprado para los primeros 5 días

# Filtrar las transacciones para los primeros 5 días
fecha_inicial = transaction_df['fecha'].min()  # Fecha inicial que corresponde al primer día
fecha_final = fecha_inicial + pd.DateOffset(days=4)  # Fecha final que corresponde al quinto día
transaction_df_first_5_days = transaction_df[(transaction_df['fecha'] >= fecha_inicial) & (transaction_df['fecha'] <= fecha_final)]

# Convertir la columna 'productos_carro' de cadena a lista
transaction_df_first_5_days['productos_carro'] = transaction_df_first_5_days['productos_carro'].apply(eval)

# Explotar la lista de productos en filas separadas
transaction_df_first_5_days = transaction_df_first_5_days.explode('productos_carro')

# Agrupar por producto y calcular el precio promedio por producto por carrito
precio_promedio_por_producto = transaction_df_first_5_days.groupby('productos_carro')['total_pagado_usd'].mean().round(2)

print(precio_promedio_por_producto)

In [ ]:
# Obtener los top 5 vendedores con mayor cantidad de ítems en inventario

# Agrupar por vendedor y calcular la suma de inventario para cada vendedor
inventario_por_vendedor = inventory_df.groupby('id_vendedor')['inventario'].sum()

# Obtener los 5 vendedores con la mayor cantidad de ítems en inventario
top_5_vendedores = inventario_por_vendedor.nlargest(5)

print(top_5_vendedores)

In [ ]:
# Obtener los top 5 vendedores que vendieron la mayor cantidad de ítems

# Explotar la lista de productos en filas separadas
transaction_df = transaction_df.explode('productos_carro')

# Agrupar por vendedor y contar la cantidad de ítems vendidos por cada vendedor
cantidad_items_por_vendedor = transaction_df.groupby('id_usuario')['productos_carro'].count()

# Obtener los 5 vendedores que vendieron la mayor cantidad de ítems
top_5_vendedores = cantidad_items_por_vendedor.nlargest(5)

print(top_5_vendedores)

En este código, hemos agregado un ciclo que verifica si hay suficientes productos en el inventario para cada producto en cada transacción. Si un producto no está disponible en el inventario en la fecha de la transacción o si el inventario es igual o menor a 0, se considera que la transacción no tiene sentido. Las transacciones sin sentido se registran en el diccionario transacciones_sin_sentido, con el usuario como clave y su lista de 'id_orden' como valor. Este diccionario contendrá las transacciones que no tienen sentido según el criterio de inventario insuficiente.

In [ ]:
# SKIP THIS APPROACH - IT´s TAKILG TOO LONG
# Crear un diccionario para almacenar transacciones sin sentido
# transacciones_sin_sentido = {}

# # Iterar a través de las transacciones
# for index, transaccion in transaction_df.iterrows():
#     fecha_transaccion = transaccion['fecha']
#     usuario = transaccion['id_usuario']
#     orden = transaccion['id_orden']
#     producto = transaccion['productos_carro']

#     # Verificar si hay suficientes productos en el inventario para la transacción
#     for producto_comprado in producto:
#         producto_existente = inventory_df[(inventory_df['id_producto'] == producto_comprado) & (inventory_df['fecha'] == fecha_transaccion)]
#         if producto_existente.empty or producto_existente['inventario'].values[0] <= 0:
#             # La transacción no tiene sentido
#             if usuario not in transacciones_sin_sentido:
#                 transacciones_sin_sentido[usuario] = []
#             transacciones_sin_sentido[usuario].append(orden)

# # Imprimir el diccionario de transacciones sin sentido
# print(transacciones_sin_sentido)

In [ ]:

# Combinar datos de transacciones con datos de inventario basándose en 'productos_carro' y 'fecha'
merged_df = transaction_df.merge(inventory_df, left_on=['productos_carro', 'fecha'], right_on=['id_producto', 'fecha'], how='left')

In [ ]:

# Filtrar transacciones con datos de inventario faltantes o donde 'inventario' es <= 0
transacciones_invalidas = merged_df[(merged_df['inventario'].isna()) | (merged_df['inventario'] <= 0)]

In [ ]:
# Agrupar transacciones inválidas por 'id_usuario' y recopilar los valores de 'id_orden'
transacciones_invalidas_agrupadas = transacciones_invalidas.groupby('id_usuario')['id_orden'].apply(list).reset_index()

In [ ]:

# Crear un diccionario donde el usuario es la clave y sus órdenes inválidas son los valores
transacciones_sin_sentido = dict(zip(transacciones_invalidas_agrupadas['id_usuario'], transacciones_invalidas_agrupadas['id_orden']))

In [ ]:
# Imprimir el diccionario de transacciones inválidas
print(transacciones_sin_sentido)